# Обучение языковой модели

По ранее собранным данным поисковых запросов обучается языковая модель FastText, которая в дальнейшем используется в векторном поиске.

In [8]:
import sys
sys.path.append('src/')

In [2]:
from src.tokenizer import Tokenizer

In [3]:
import pandas as pd
from gensim.models import FastText
from random import shuffle
from tqdm.auto import tqdm

tqdm.pandas()

__Загрузка данных__

In [4]:
tokenizer = Tokenizer()

In [5]:
data = pd.read_parquet('data/keywords.parquet')
data

,keyword,cnt
0,0 0 2. почему вы не богатые если вы такие умны...,2
1,0 00 18 10 intro hp victus 1,16
2,0 0060 0,44
3,0 024,28
4,0 092 s30 070,1
...,...,...
3409680,𝐑𝟵 𝟕𝟵𝟎𝟎𝐗,8
3409681,𝐒𝐨𝐧𝐲 𝐏𝐥𝐚𝐲𝐒𝐭𝐚𝐭𝐢𝐨𝐧 5,57
3409682,𝙉𝙤𝙠𝙞𝙖 6300 4 𝙜,10
3409683,𝙲𝚌𝚏𝚎250,33


__Токенизация__

In [ ]:
data['tokens'] = data.keyword.progress_apply(tokenizer.tokenize)

  0%|          | 0/3409685 [00:00<?, ?it/s]

__Фильтрация пустых запросов__

In [10]:
data = data[data.tokens.apply(lambda x: len(x) > 0)]

__Формирование корпуса токенов__

In [12]:
corpus = []
for tokens, cnt in tqdm(data[['tokens', 'cnt']].values):
    corpus += [tokens] * cnt
shuffle(corpus)
len(corpus)

  0%|          | 0/3409654 [00:00<?, ?it/s]

624982722

__Обучение модели__

In [14]:
model = FastText(
    sentences=corpus,
    vector_size=100,
    window=3,
    min_count=10,
    workers=32,
    epochs=10,
    negative=3,
    sg=1
)

In [15]:
model.save('models/lm')

__Проверка модели на адекватность__

In [26]:
model.wv.most_similar('xiomi')

[('xiaomi', 0.9255608916282654),
 ('xioaomi', 0.8939005136489868),
 ('xiaomixiaomi', 0.885464072227478),
 ('wiaomi', 0.8630273938179016),
 ('xiami', 0.85608971118927),
 ('xiaommi', 0.8536121249198914),
 ('сяом', 0.8519089818000793),
 ('xiiaomi', 0.8509979844093323),
 ('xiaomj', 0.8475507497787476),
 ('vxiaomi', 0.845581591129303)]

In [42]:
model.wv.distance('xiaomi', 'смартфон'), model.wv.distance('iphone', 'смартфон')

(0.7063806354999542, 0.4842911958694458)